# Example 4.1

This notebook shows Example 4.1 from the book showing how to use SQL on a US Flights Dataset dataset.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Example 4.1').getOrCreate()

Define a UDF to convert the date format into a legible format.

*Note*: the date is a string with year missing, so it might be difficult to do any queries using SQL `year()` function

In [3]:
def to_date_format_udf(date_str):
    l = [char for char in date_str]
    return "".join(l[0:2]) + "/" + "".join(l[2:4])+ " " + "".join(l[4:6]) + ":" + "".join(l[6:])

to_date_format_udf("02190925")

'02/19 09:25'

In [4]:
spark.udf.register("to_date_format_udf",to_date_format_udf,StringType())

<function __main__.to_date_format_udf(date_str)>

Read our US departure flight data

In [6]:
df = (spark.read
     .format("csv")
     .option("header","true")
     .schema("date STRING,delay INT, distance INT, origin STRING, destination STRING")
     .option("path","/home/karthik/SparkCourse/pyspark notebooks/data/departuredelays.csv")
     .load())

In [7]:
df.show(10,False)

+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
|01030605|0    |602     |ABE   |ATL        |
|01041243|10   |602     |ABE   |ATL        |
|01040605|28   |602     |ABE   |ATL        |
|01051245|88   |602     |ABE   |ATL        |
|01050605|9    |602     |ABE   |ATL        |
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [8]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [18]:
df.count()

1391578

Test our UDF

In [13]:
df.select("date",expr("to_date_format_udf(date) as date_formatted")).show(10,False)

+--------+--------------+
|date    |date_formatted|
+--------+--------------+
|01011245|01/01 12:45   |
|01020600|01/02 06:00   |
|01021245|01/02 12:45   |
|01020605|01/02 06:05   |
|01031245|01/03 12:45   |
|01030605|01/03 06:05   |
|01041243|01/04 12:43   |
|01040605|01/04 06:05   |
|01051245|01/05 12:45   |
|01050605|01/05 06:05   |
+--------+--------------+
only showing top 10 rows



Create a temporary view to which we can issue SQL queries

In [15]:
df.createOrReplaceTempView("us_flight_delays_tbl")

Cache Table so queries are expedient

In [16]:
spark.sql("CACHE TABLE us_flight_delays_tbl")

DataFrame[]

Convert all `date` to `date_fm` so it's more eligible

Note: we are using UDF to convert it on the fly. 

In [17]:
spark.sql("SELECT *,date,to_date_format_udf(date) FROM us_flight_delays_tbl LIMIT 10").show(truncate=False)

+--------+-----+--------+------+-----------+--------+------------------------+
|date    |delay|distance|origin|destination|date    |to_date_format_udf(date)|
+--------+-----+--------+------+-----------+--------+------------------------+
|01011245|6    |602     |ABE   |ATL        |01011245|01/01 12:45             |
|01020600|-8   |369     |ABE   |DTW        |01020600|01/02 06:00             |
|01021245|-2   |602     |ABE   |ATL        |01021245|01/02 12:45             |
|01020605|-4   |602     |ABE   |ATL        |01020605|01/02 06:05             |
|01031245|-4   |602     |ABE   |ATL        |01031245|01/03 12:45             |
|01030605|0    |602     |ABE   |ATL        |01030605|01/03 06:05             |
|01041243|10   |602     |ABE   |ATL        |01041243|01/04 12:43             |
|01040605|28   |602     |ABE   |ATL        |01040605|01/04 06:05             |
|01051245|88   |602     |ABE   |ATL        |01051245|01/05 12:45             |
|01050605|9    |602     |ABE   |ATL        |01050605

In [19]:
spark.sql("SELECT COUNT(*) FROM us_flight_delays_tbl").show()

+--------+
|count(1)|
+--------+
| 1391578|
+--------+



### Query 1:

 Find out all flights whose distance between origin and destination is greater than 1000 

In [20]:
spark.sql("""SELECT origin
                  , destination
                  , distance 
               FROM us_flight_delays_tbl 
              WHERE distance > 1000 
            ORDER BY distance DESC""").show(truncate=False)

+------+-----------+--------+
|origin|destination|distance|
+------+-----------+--------+
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
+------+-----------+--------+
only showing top 20 rows



Getting the same results as above using DataFrames

In [21]:
(df
 .select("origin","destination","distance")
 .filter(col("distance")>1000)
 .orderBy("distance", ascending=False)
 .show(truncate=False))

+------+-----------+--------+
|origin|destination|distance|
+------+-----------+--------+
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
|HNL   |JFK        |4330    |
+------+-----------+--------+
only showing top 20 rows



### Query 2:

 Find out all flights with 2 hour delays between San Francisco and Chicago  

In [23]:
spark.sql("""
SELECT origin
     , destination
     , delay
  FROM us_flight_delays_tbl
 WHERE origin = 'SFO'
   AND destination = 'ORD'
   AND delay > 120
 ORDER BY delay DESC""").show()

+------+-----------+-----+
|origin|destination|delay|
+------+-----------+-----+
|   SFO|        ORD| 1638|
|   SFO|        ORD|  396|
|   SFO|        ORD|  326|
|   SFO|        ORD|  320|
|   SFO|        ORD|  297|
|   SFO|        ORD|  296|
|   SFO|        ORD|  279|
|   SFO|        ORD|  274|
|   SFO|        ORD|  266|
|   SFO|        ORD|  258|
|   SFO|        ORD|  225|
|   SFO|        ORD|  223|
|   SFO|        ORD|  215|
|   SFO|        ORD|  203|
|   SFO|        ORD|  197|
|   SFO|        ORD|  196|
|   SFO|        ORD|  193|
|   SFO|        ORD|  190|
|   SFO|        ORD|  189|
|   SFO|        ORD|  184|
+------+-----------+-----+
only showing top 20 rows



DataFrame Equivalent Query

In [24]:
(df
 .select("origin","destination","delay")
 .filter(expr("origin == 'SFO' AND destination = 'ORD' AND delay > 120"))
 .orderBy("delay",ascending=False)
 .show())

+------+-----------+-----+
|origin|destination|delay|
+------+-----------+-----+
|   SFO|        ORD| 1638|
|   SFO|        ORD|  396|
|   SFO|        ORD|  326|
|   SFO|        ORD|  320|
|   SFO|        ORD|  297|
|   SFO|        ORD|  296|
|   SFO|        ORD|  279|
|   SFO|        ORD|  274|
|   SFO|        ORD|  266|
|   SFO|        ORD|  258|
|   SFO|        ORD|  225|
|   SFO|        ORD|  223|
|   SFO|        ORD|  215|
|   SFO|        ORD|  203|
|   SFO|        ORD|  197|
|   SFO|        ORD|  196|
|   SFO|        ORD|  193|
|   SFO|        ORD|  190|
|   SFO|        ORD|  189|
|   SFO|        ORD|  184|
+------+-----------+-----+
only showing top 20 rows



### Query 3:

A more complicated query in SQL, let's label all US flights originating from airports with _high_, _medium_, _low_, _no delays_, regardless of destinations.

In [25]:
spark.sql("""
SELECT origin
     , destination
     , delay
     , CASE WHEN delay >= 360 THEN 'Very Long Delays'
            WHEN delay >= 120 AND delay < 360 THEN 'Long Delays'
            WHEN delay >= 60 AND delay < 120 THEN 'Short Delays'
            WHEN delay > 0 AND delay < 60 THEN 'Very Short Delays'
            ELSE 'Early' END AS flight_delays
  FROM us_flight_delays_tbl
 ORDER BY origin, delay DESC""").show()

+------+-----------+-----+-------------+
|origin|destination|delay|flight_delays|
+------+-----------+-----+-------------+
|   ABE|        ATL|  333|  Long Delays|
|   ABE|        ATL|  305|  Long Delays|
|   ABE|        ATL|  275|  Long Delays|
|   ABE|        ATL|  257|  Long Delays|
|   ABE|        ATL|  247|  Long Delays|
|   ABE|        DTW|  247|  Long Delays|
|   ABE|        ORD|  219|  Long Delays|
|   ABE|        ATL|  211|  Long Delays|
|   ABE|        DTW|  197|  Long Delays|
|   ABE|        ORD|  192|  Long Delays|
|   ABE|        ATL|  180|  Long Delays|
|   ABE|        DTW|  173|  Long Delays|
|   ABE|        ATL|  165|  Long Delays|
|   ABE|        ATL|  159|  Long Delays|
|   ABE|        ORD|  159|  Long Delays|
|   ABE|        ATL|  158|  Long Delays|
|   ABE|        DTW|  151|  Long Delays|
|   ABE|        ATL|  127|  Long Delays|
|   ABE|        DTW|  121|  Long Delays|
|   ABE|        DTW|  118| Short Delays|
+------+-----------+-----+-------------+
only showing top

Equivalent DataFrame Query

In [28]:
(df
.select("origin","destination","delay")
.withColumn("flight_delays",
            expr("CASE WHEN delay >= 360 THEN 'Very Long Delays' "
            +"WHEN delay >= 120 AND delay < 360 THEN 'Long Delays' "
            +"WHEN delay >= 60 AND delay < 120 THEN 'Short Delays' "
            +"WHEN delay > 0 AND delay < 60 THEN 'Very Short Delays' "
            +"ELSE 'Early' END AS flight_delays"))
.orderBy(asc("origin"),desc("delay"))
.show())

+------+-----------+-----+-------------+
|origin|destination|delay|flight_delays|
+------+-----------+-----+-------------+
|   ABE|        ATL|  333|  Long Delays|
|   ABE|        ATL|  305|  Long Delays|
|   ABE|        ATL|  275|  Long Delays|
|   ABE|        ATL|  257|  Long Delays|
|   ABE|        ATL|  247|  Long Delays|
|   ABE|        DTW|  247|  Long Delays|
|   ABE|        ORD|  219|  Long Delays|
|   ABE|        ATL|  211|  Long Delays|
|   ABE|        DTW|  197|  Long Delays|
|   ABE|        ORD|  192|  Long Delays|
|   ABE|        ATL|  180|  Long Delays|
|   ABE|        DTW|  173|  Long Delays|
|   ABE|        ATL|  165|  Long Delays|
|   ABE|        ATL|  159|  Long Delays|
|   ABE|        ORD|  159|  Long Delays|
|   ABE|        ATL|  158|  Long Delays|
|   ABE|        DTW|  151|  Long Delays|
|   ABE|        ATL|  127|  Long Delays|
|   ABE|        DTW|  121|  Long Delays|
|   ABE|        DTW|  118| Short Delays|
+------+-----------+-----+-------------+
only showing top